# Simple xgboost hyper-parameter tuning using hyperopt

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X_concat,y, 
                                                    test_size=0.1, 
                                                    random_state=42, stratify=y)

In [ ]:
import pandas as pd, numpy as np
from sklearn.metrics import accuracy_score
import xgboost as xgb
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import train_test_split


random_state = 42

X_train, X_cv, y_train, y_cv = train_test_split(X_concat,
                                                y, 
                                                test_size=0.1, 
                                                random_state=random_state, 
                                                stratify=y)


def objective(space):

    clf = xgb.XGBClassifier(**space)

    eval_set  = [(X_cv, y_cv)]

    clf.fit(X_train, y_train,
            eval_set=eval_set,
            early_stopping_rounds=30)

    pred = clf.predict(X_cv)
    acc = accuracy_score(y_cv, pred)
    print "SCORE:", acc

    return{'loss':1-acc, 'status': STATUS_OK }


space ={
        'n_estimators' :hp.choice('n_estimators', np.arange( 100, 1000, dtype=int)),
        'max_depth': hp.choice('max_depth', np.arange(1, 10, dtype=int)),
        'min_child_weight': hp.quniform ('x_min_child', 1, 10, 1),
        'subsample': hp.uniform ('x_subsample', 0.8, 1),
        'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        'eval_metric': 'error',
        'objective': 'binary:logistic',
        'nthread' : -1,
        'silent' : 0,
        'random_state': random_state
    }


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print best

# Parameters for gridsearch to be used

In [2]:
param_dist_rigid = {
    'clf_xgb':
        { 
            'clf_xgb__n_estimators': [100, 250,500,750, 1000],
            'clf_xgb__learning_rate': [0.01, 0.03, 0.05, 0.07],
            'clf_xgb__subsample': [0.3, 0.5, 0.7, 0.9],
            'clf_xgb__max_depth': [3, 4, 5,8],
            'clf_xgb__gamma': [0.5,0.7,0.8,1],
            'clf_xgb__colsample_bytree': [0.5, 0.7, 0.9],
            'clf_xgb__seed': [42]
         }
}